In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [67]:
#test
#import sys
#print(sys.version)
#import numpy

In [13]:
%matplotlib inline
import os
import geopandas as gp
from shapely.geometry import Polygon
import matplotlib
import matplotlib.pyplot as plt
import expo_nrml as nrml


### define a ROI

In [77]:
lat_point_list = [-33.2,-33.0 , -33.0, -33.2, -33.2]
lon_point_list = [-71.8, -71.8,-71.4,-71.4,-71.8]

roi_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
roi = gp.GeoDataFrame(index=[0], crs=crs, geometry=[roi_geom])       

In [143]:
roi_geom.wkt

'POLYGON ((-71.8 -33.2, -71.8 -33, -71.40000000000001 -33, -71.40000000000001 -33.2, -71.8 -33.2))'

### Init model from file (geopackage)

In [90]:
rj = roi.to_json()
#rj
gs = gp.GeoSeries(rj)


0    {"type": "FeatureCollection", "features": [{"i...
dtype: object

In [50]:
'''
read exposure model from a (geopackage) file, filter and harmonise 
the columns in order to have for each geocell the following:
index: integer unique identifier
gc_id: string, unique identifier
name: string, name
geometry: polygon
a number of columns follow, each describing a single taxonomic type

returns the model as geopandas dataframe and the list of taxonomic types
'''
def read_model(input_file):
    #init model
    res = gp.read_file(input_file,encoding = 'utf-8')
    taxonomies = res.keys()[res.dtypes=='float64']
    cols = ['GID_3','NAME_3','geometry',*taxonomies.values]
    out = res[cols].reset_index()
    out.columns = ['index','gc_id','name','geometry',*taxonomies.values]
    return [out,taxonomies]

'''
extract a part of the model by doing a spatial query on the geopandas df
return a sub-portion of a model based on a ROI and a query mode: 
'within': returns the geometries that are completely inside the ROI
'intersects': returns the geometries that are intersecting the ROI
'''
def queryModelfromRoi(mod,roi,mode='within'):
    #modes=['within','intersects']
    r = roi.geometry.iloc[0]
    if (mode=='within'):
        res=mod[mod.within(r)]
    elif(mode=='intersects'):
        res=mod[mod.intersects(r)]
    else:
        raise Exception ('getModelfromRoi: unknown mode')
    return(res)

'''
Export geopandas dataframe as GeoJson file
'''
def exportGeoJson(dataframe, filename):
    # file has to be first deleted
    # because driver does not support overwrite ! 
    try: 
        os.remove(filename)
    except OSError:
        pass
    dataframe.to_file(filename, driver='GeoJSON')
    return (0)

def exportNrml05(dataframe, filename, metadata, dicts,taxonomies):
    xml_string = nrml.write_nrml05_expo(dataframe,metadata,dicts,taxonomies,filename)
    return (0)

#test
#sub = getModelfromRoi(model,roi,'intersects')
#print(sub)
#sub.plot()

### Main 

In [57]:
schema = 'SARA_v1.0'

In [58]:
def main(roi,schema):
    
    #init file path and file names according to specific schema.
    #currently only the SARA v1.0 schema is supported

    if (schema == 'SARA_v1.0'):
        path_expo_dict ="/Users/pittore/Documents/workspace/RIESGOS/assetmaster/model_SARA_v1.0"
        path_metadatefile = "/Users/pittore/Documents/workspace/RIESGOS/assetmaster/model_SARA_v1.0"
        path_infile = "/Users/pittore/Documents/workspace/RIESGOS/assetmaster/model_SARA_v1.0"
        path_outfile = "/Users/pittore/Documents/workspace/RIESGOS/assetmaster/output"
    else:
        raise Exception ("schema {} not supported".format(schema))

    in_file = "chile_sara_v1.0_data.gpkg"
    dict_file = "chile_sara_v1.0_prop.csv"
    metadata_file = "chile_sara_v1.0_meta.json"
    out_file_xml = "query_output.nrml"
    out_file_geojson = 'query_output.geojson'

    #read the exposure model metadata
    metadata = nrml.read_metadata(os.path.join(path_metadatefile,metadata_file))

    #get a dataframe with the basic properties of the buildings
    dicts = nrml.load_expo_dicts(os.path.join(path_expo_dict,dict_file))

    #read taxonomies from the dict file
    btypes = dicts.btype
    #check that the asset taxonomies match
    if not (set(taxonomies) <= set(btypes)):
        raise Exception ("taxonomies do not match")

    #read model from file 
    model,taxonomies = read_model(os.path.join(path_infile,in_file))

    #spatial query
    query_res = queryModelfromRoi(model,roi,'intersects')

In [48]:
output_geojson = os.path.join(path_outfile,out_file_geojson)
exportGeoJson(query_res,output_geojson)

0

In [51]:
output_xml = os.path.join(path_outfile,out_file_xml)
exportNrml05(query_res, output_xml, metadata, dicts,taxonomies)

0

In [158]:
query_res = queryModelfromRoi(model,roi,'intersects')
query_res.empty

False

In [ ]:
import folium
m = folium.Map([-33.08,-71.6], zoom_start=8, tiles='cartodbpositron')
folium.GeoJson(polygon).add_to(m)
folium.GeoJson(res.boundary,
    style_function=lambda feature: {
        'fillColor': '#00FF00',
        'color' : '#32CD32',
        'weight' : 1,
        'fillOpacity' : 0.5,
        }).add_to(m)
folium.LatLngPopup().add_to(m)
m

## Final Python implementation

In [148]:
#!/usr/bin/env/python

'''
Assetmaster
-----------------
Command line program to query exposure data from a database/file.
'''

import argparse
import os
import geopandas as gp
from shapely.geometry import Polygon
import matplotlib
import matplotlib.pyplot as plt
import expo_nrml as nrml

class Main():
    '''
    Main class to execute
    '''
    def __init__(self, args):
        self.folder = os.path.dirname(__file__)

        # command line arguments
        self.lonmin = args.lonmin
        self.lonmax = args.lonmax
        self.latmin = args.latmin
        self.latmax = args.latmax
        self.schema = args.schema
        self.assettype = args.assettype
        
        #i/o settings
        self.path_expo_dict = self.folder
        self.path_metadatefile = self.folder
        self.path_infile = self.folder
        self.in_file = self.folder
        self.dict_file = self.folder
        self.metadata_file = self.folder
        self.path_outfile = os.path.join(self.folder,"output")
        self.out_file_xml = "query_output.nrml"
        self.out_file_geojson = 'query_output.geojson'
        
        self.roi = None
        self.metadata = None
        self.dicts = None 
        self.taxonomies = None

        #list of supported schemas. 
        #TODO: automatically parse them from a given folder
        self.supported_schemas = ['SARA_v1.0']

        # in case there is some deaggregation necessary
        # precision
        self.precision_lon = 0
        self.precision_lat = 0

        # result
        self.query_result = None

    def _compute_roi(self):
        lat_point_list = [self.latmin,self.latmax ,self.latmax, self.latmin,self.latmin]
        lon_point_list = [self.lonmin, self.lonmin,self.lonmax,self.lonmax,self.lonmin]
        roi_geom = Polygon(zip(lon_point_list, lat_point_list))
        crs = {'init': 'epsg:4326'}
        self.roi = gp.GeoDataFrame(index=[0], crs=crs, geometry=[roi_geom])       

    def _check_schema(self):
        return(set([self.schema]) <= set(self.supported_schemas))

    def _check_longitude(self):
        '''If there is a longitude > 180 than it should be converted'''
        if self.lonmin > 180:
            self.lonmin = Main._convert_360(self.lonmin)
        if self.lonmax > 180:
            self.lonmax = Main._convert_360(self.lonmax)
            
    @staticmethod
    def _convert_360(lon):
        '''
        convert a longitude specified with 180+
        '''
        return lon-360
    
        
    def read_model(self,input_file):
        '''
        read exposure model from a (geopackage) file, filter and harmonise 
        the columns in order to have for each geocell the following:
        index: integer unique identifier
        gc_id: string, unique identifier
        name: string, name
        geometry: polygon
        a number of columns follow, each describing a single taxonomic type

        returns the model as geopandas dataframe and the list of taxonomic types
        '''
        #init model
        #input_file = 'schemas/SARA_v1.0/SARA_v1.0_data.gpkg'
        res = gp.read_file(input_file,encoding = 'utf-8')
        taxonomies = res.keys()[res.dtypes=='float64']
        cols = ['GID_3','NAME_3','geometry',*taxonomies.values]
        out = res[cols].reset_index()
        out.columns = ['index','gc_id','name','geometry',*taxonomies.values]
        return [out,taxonomies]
    
    def queryModelfromRoi(self,mod,roi,mode='within'):
        '''
        extract a part of the model by doing a spatial query on the geopandas df
        return a sub-portion of a model based on a ROI and a query mode: 
        'within': returns the geometries that are completely inside the ROI
        'intersects': returns the geometries that are intersecting the ROI
        '''
        #modes=['within','intersects']
        r = roi.geometry.iloc[0]
        if (mode=='within'):
            res=mod[mod.within(r)]
        elif(mode=='intersects'):
            res=mod[mod.intersects(r)]
        else:
            raise Exception ('getModelfromRoi: unknown mode')
        return(res)
    
    def _exportGeoJson(self, dataframe, filename):
        '''
        Export geopandas dataframe as GeoJson file
        '''
        # file has to be first deleted
        # because driver does not support overwrite ! 
        try: 
            os.remove(filename)
        except OSError:
            pass
        dataframe.to_file(filename, driver='GeoJSON')
        return (0)

    def _exportNrml05(self, dataframe, filename, metadata, dicts,taxonomies):
        '''
        Export geopandas dataframe as nrml file
        '''
        xml_string = nrml.write_nrml05_expo(dataframe,metadata,dicts,taxonomies,filename)
        return (0)

    def _write_outputs(self):
        '''
        Export query result as nrml and geojson files
        '''
        output_geojson = os.path.join(self.path_outfile,self.out_file_geojson)
        self._exportGeoJson(self.query_result,output_geojson)
        output_xml = os.path.join(self.path_outfile,self.out_file_xml)
        self._exportNrml05(self.query_result, output_xml, self.metadata, 
                           self.dicts,self.taxonomies)
    
    def run(self):
        '''
        Method to:
        - connect with the database
        - query the database
        - do some deaggregation if necessary
        - write the output
        '''

        self._check_longitude()
        self._compute_roi()
        
        if (self._check_schema()):
            foldername = os.path.join(self.folder,"schemas/{}".format(self.schema))
            self.path_expo_dict = foldername
            self.path_metadatefile = foldername
            self.path_infile = foldername
            self.in_file = "{}_data.gpkg".format(self.schema)
            self.dict_file = "{}_prop.csv".format(self.schema)
            self.metadata_file = "{}_meta.json".format(self.schema)
        else:
            raise Exception ("schema {} not supported".format(self.schema))

        #read the exposure model metadata
        self.metadata = nrml.read_metadata(os.path.join(self.path_metadatefile,self.metadata_file))

        #read model from file 
        in_file = os.path.join(self.path_infile,self.in_file)
        
        self.model,self.taxonomies = self.read_model(in_file)#'schemas/SARA_v1.0/SARA_v1.0_data.gpkg')

        #print(self.taxonomies)
        #get a dataframe with the basic properties of the buildings
        self.dicts = nrml.load_expo_dicts(os.path.join(self.path_expo_dict,self.dict_file))

        #read taxonomies from the dict file
        btypes = self.dicts.btype

        #check that the asset taxonomies match
        if not (set(self.taxonomies) <= set(btypes)):
            raise Exception ("taxonomies do not match")

        #spatial query
        print (self.roi)
        self.query_result = self.queryModelfromRoi(self.model,self.roi,'intersects')

        self._write_outputs()
        return (0)


    @classmethod
    def create_with_arg_parser(cls):
        '''
        Creates an arg parser and uses that to create the Main class
        '''
        arg_parser = argparse.ArgumentParser(
            description='''Program to query an exposure model from a database/file'''
        )
        arg_parser.add_argument(
            'lonmin',
            help='Region Of Interest: Minimal longitude',
            type=float,
            default=-71.8)
        arg_parser.add_argument(
            'lonmax',
            help='Region Of Interest: Maximal longitude',
            type=float,
            default=-71.4)
        arg_parser.add_argument(
            'latmin',
            help='Region Of Interest: Minimal latitude',
            type=float,
            default=-33.2)
        arg_parser.add_argument(
            'latmax',
            help='Region Of Interest: Maximal latitude',
            type=float,
            default=-33.0)
        arg_parser.add_argument(
            'schema',
            help='Exposure/Vulnerability Schema',
            type=str,
            default="SARA_v1.0")
        arg_parser.add_argument(
            'assettype',
            help='Type of exposed assets',
            type=str,
            default='res')
        args = arg_parser.parse_args()

        return cls(args)

if __name__ == '__main__':
    Main.create_with_arg_parser().run()

In [155]:
{assettype='res', latmax=-33.0, latmin=-33.2, lonmax=-71.4, lonmin=-71.8, schema='SARA_v1.0'}
#Main()

SyntaxError: invalid syntax (<ipython-input-155-3b5053ae029f>, line 1)